# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({'user_id': ['u1','u2','u3'], 
                     'recommended_list':    [[143, 156, 1134, 991, 27],
                                             [1543, 3345, 533, 11, 43],
                                             [156, 3345, 10, 15, 1134]],
                     
                      'bought_list':        [[156, 27], [11, 43], [1]],
                     
                      'prices_recommended': [[5, .5, 8., 2., 1.5],
                                             [8., 4.4, 2., 2., 4],
                                             [.5, 4.4, 4., 6., 8.]],
                     
                      'prices_bought':      [[.5, 1.5], [2., 4], [np.nan]]
                     })
data

,user_id,recommended_list,bought_list,prices_recommended,prices_bought
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[5, 0.5, 8.0, 2.0, 1.5]","[0.5, 1.5]"
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[8.0, 4.4, 2.0, 2.0, 4]","[2.0, 4]"
2,u3,"[156, 3345, 10, 15, 1134]",[1],"[0.5, 4.4, 4.0, 6.0, 8.0]",[nan]


## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    flags = np.isin(np.array(recommended_list)[:k], np.array(bought_list))
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):

    flags = np.isin(np.array(recommended_list)[:k], np.array(bought_list))
    return (flags * np.array(prices_recommended)[:k]).sum() / np.array(prices_bought).sum()

In [4]:
data['recall_at_5'] = data.apply(lambda x: recall_at_k(x['recommended_list'], x['bought_list'], k=5), axis=1)
mean_recall = data['recall_at_5'].mean()
print(f'{mean_recall=:.4f}')

data['money_recall_at_4'] = data.apply(lambda x: money_recall_at_k(x['recommended_list'], 
                                                                   x['bought_list'], 
                                                                   x['prices_recommended'], 
                                                                   x['prices_bought'], 
                                                                   k=4), axis=1)
money_mean_recall = data['money_recall_at_4'].mean()
print(f'{money_mean_recall=:.4f}')

data

mean_recall=0.6667
money_mean_recall=0.2917


,user_id,recommended_list,bought_list,prices_recommended,prices_bought,recall_at_5,money_recall_at_4
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[5, 0.5, 8.0, 2.0, 1.5]","[0.5, 1.5]",1.0,0.250000
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[8.0, 4.4, 2.0, 2.0, 4]","[2.0, 4]",1.0,0.333333
2,u3,"[156, 3345, 10, 15, 1134]",[1],"[0.5, 4.4, 4.0, 6.0, 8.0]",[nan],0.0,NaN


## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [5]:
def mrr_at_k(recommended_list, bought_list, k):

    flags = np.isin(np.array(recommended_list)[:k], np.array(bought_list))
    if flags.sum() == 0:
        return 0

    rank = np.where(flags == True)[0][0] + 1 
    return 1 / rank

In [6]:
data['mrr_at_k'] = data.apply(lambda x: mrr_at_k(x['recommended_list'], 
                                                 x['bought_list'], 
                                                 k=5), axis=1)
mean_mrr = data['mrr_at_k'].mean()
print(f'{mean_mrr=:.4f}')

data

mean_mrr=0.2500


,user_id,recommended_list,bought_list,prices_recommended,prices_bought,recall_at_5,money_recall_at_4,mrr_at_k
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[5, 0.5, 8.0, 2.0, 1.5]","[0.5, 1.5]",1.0,0.250000,0.50
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[8.0, 4.4, 2.0, 2.0, 4]","[2.0, 4]",1.0,0.333333,0.25
2,u3,"[156, 3345, 10, 15, 1134]",[1],"[0.5, 4.4, 4.0, 6.0, 8.0]",[nan],0.0,NaN,0.00


## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [7]:
def ndcg_at_k(recommended_list, bought_list, k=5):

    dcg_at_k, idcg_at_k = 0, 0
    flags = np.isin(np.array(recommended_list)[:k], np.array(bought_list))
    if sum(flags) == 0: 
        return 0    

    for i in range(len(flags)):
        if flags[i]:
            dcg_at_k += 1 / np.log2(i+2) 
        idcg_at_k += 1 / np.log2(i+2)    
    if idcg_at_k == 0:                   
        ndcg_at_k = 0                    
    else:
        ndcg_at_k = dcg_at_k/idcg_at_k

    return ndcg_at_k
    

In [8]:
data['ndcg_at_5'] = data.apply(lambda x: ndcg_at_k(x['recommended_list'], x['bought_list'], k=4), axis=1)
mean_ndcg = data['ndcg_at_5'].mean()
print(f'{mean_ndcg=:.4f}')

data

mean_ndcg=0.1381


,user_id,recommended_list,bought_list,prices_recommended,prices_bought,recall_at_5,money_recall_at_4,mrr_at_k,ndcg_at_5
0,u1,"[143, 156, 1134, 991, 27]","[156, 27]","[5, 0.5, 8.0, 2.0, 1.5]","[0.5, 1.5]",1.0,0.250000,0.50,0.246302
1,u2,"[1543, 3345, 533, 11, 43]","[11, 43]","[8.0, 4.4, 2.0, 2.0, 4]","[2.0, 4]",1.0,0.333333,0.25,0.168128
2,u3,"[156, 3345, 10, 15, 1134]",[1],"[0.5, 4.4, 4.0, 6.0, 8.0]",[nan],0.0,NaN,0.00,0.000000
